# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [80]:
weather_df = pd.read_csv('weather_data.csv')
weather_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness %,Wind Speed (mph)
0,0,rubtsovsk,51.5000,81.2500,48.560,51,100,2.99
1,1,pangkalanbuun,-2.6833,111.6167,80.204,84,95,1.37
2,2,leningradskiy,69.3833,178.4167,19.436,92,100,10.70
3,3,new norfolk,-42.7826,147.0587,62.816,55,75,5.14
4,4,kichera,55.9375,110.0997,38.876,62,100,0.72
...,...,...,...,...,...,...,...,...
1305,1305,lar,26.2039,83.9717,84.362,34,0,3.22
1306,1306,bajil,15.0584,43.2851,84.992,56,96,1.86
1307,1307,pau dos ferros,-6.1125,-38.2063,77.072,70,100,1.19
1308,1308,lampazos de naranjo,27.0167,-100.5167,66.362,76,100,4.30


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [81]:
gmaps.configure(api_key=g_key)

In [82]:
locations = weather_df[["Latitude", "Longitude"]]
humid = weather_df['Humidity']
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
#temp below 80 
#100% cloudy
#wind speed over 10mph

hotel_df = weather_df[weather_df['Temperature'] < 80]
hotel_df = hotel_df[hotel_df['Cloudiness %'] == 100]
hotel_df = hotel_df[hotel_df['Wind Speed (mph)'] > 10]
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness %,Wind Speed (mph),Hotel Name
2,2,leningradskiy,69.3833,178.4167,19.436,92,100,10.70,
25,25,burgeo,47.6167,-57.6152,39.218,93,100,15.18,
117,117,lebu,-37.6167,-73.6500,56.390,91,100,10.22,
177,177,berlevag,70.8578,29.0864,33.746,72,100,12.35,
217,217,tombouctou,20.0000,-3.0000,75.560,55,100,10.27,
221,221,severo-kurilsk,50.6789,156.1250,35.528,96,100,10.18,
377,377,vao,-22.6667,167.4833,74.588,70,100,10.43,
389,389,poronaysk,49.2167,143.1167,38.300,82,100,10.14,
421,421,vakhrushev,48.9833,142.9667,35.474,97,100,10.43,
1224,1224,marystown,47.1666,-55.1483,39.056,96,100,12.25,


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [88]:
# 4. Find a balloon store near the White House.
hotel_names = []
hotels_lat = []
hotels_lng = []
for i, row in hotel_df.iterrows():
    search = "Hotel"
    coords = f"{row['Latitude']},{row['Longitude']}"
    radius = 5000
    
    # redefine params
    params = {
        "location": coords,
        "keyword": search,
        "radius": radius,
        "key": g_key
    }

    # Build URL using the Google Maps API
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Run request
    current_hotel = requests.get(base_url, params).json()
    
    if current_hotel['status'] != "ZERO_RESULTS":
   
        hotel_name = current_hotel["results"][0]["name"]
        hotel_lat = current_hotel["results"][0]["geometry"]["location"]["lat"]
        hotel_lng = current_hotel["results"][0]["geometry"]["location"]["lng"]

        hotel_names.append(hotel_name)
        hotels_lat.append(hotel_lat)
        hotels_lng.append(hotel_lng)
    else:
        hotel_names.append("NO HOTEL")

hotel_df['Hotel Name'] = hotel_names
hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temperature,Humidity,Cloudiness %,Wind Speed (mph),Hotel Name
2,2,leningradskiy,69.3833,178.4167,19.436,92,100,10.70,NO HOTEL
25,25,burgeo,47.6167,-57.6152,39.218,93,100,15.18,The Old Salt Box Co. Skinner's House Burgeo
117,117,lebu,-37.6167,-73.6500,56.390,91,100,10.22,Hotel Plaza Lebu
177,177,berlevag,70.8578,29.0864,33.746,72,100,12.35,Berlevåg camping & Appartement AS
217,217,tombouctou,20.0000,-3.0000,75.560,55,100,10.27,NO HOTEL
221,221,severo-kurilsk,50.6789,156.1250,35.528,96,100,10.18,Kak Doma
377,377,vao,-22.6667,167.4833,74.588,70,100,10.43,OURE TERA beach resort
389,389,poronaysk,49.2167,143.1167,38.300,82,100,10.14,Sever
421,421,vakhrushev,48.9833,142.9667,35.474,97,100,10.43,NO HOTEL
1224,1224,marystown,47.1666,-55.1483,39.056,96,100,12.25,Marystown Hotel & Convention Centre


In [93]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>

</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [94]:
# Add marker layer ontop of heat map
coordinates = zip(hotels_lat, hotels_lng)
coordinates = list(coordinates)
# Display figure
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))